In [15]:
import collections
import pandas as pd
import plotly.express as px

In [16]:
melon_df = pd.read_csv('melon_reviews.csv',lineterminator='\n')
del melon_df['Unnamed: 0']

In [17]:
import numpy as np
import math
bubble_data = []
viewsum,scoreavg,revcount = collections.defaultdict(int),collections.defaultdict(list),collections.defaultdict(int)

melon_dict = melon_df.to_dict('records')
artist_set = set([ele['artist'] for ele in melon_dict])
for date in range(2010,2022+1):
    for artist in artist_set:
        temp = [(ele['views'],ele['score']) for ele in melon_dict if ele['artist']==artist and str(date) in ele['date']]
        for views,score in temp:
            if not math.isnan(views):
                viewsum[artist] += views
            if not math.isnan(score):
                scoreavg[artist].append(score)
            revcount[artist]+=1
        if scoreavg[artist]:
            bubble_data.append({'artist':artist,'year':date,'review_count':revcount[artist],'views':viewsum[artist],'score':np.mean(np.array(scoreavg[artist]))})
        else:
            bubble_data.append({'artist':artist,'year':date,'review_count':revcount[artist],'views':viewsum[artist],'score':0})

            
bubble_data = pd.DataFrame(bubble_data)

In [18]:
for artist in artist_set:
    viewcount = int(bubble_data.loc[(bubble_data['year'] == max(bubble_data['year'])) & (bubble_data['artist'] == artist)]['views'])
    if viewcount and viewcount < 1000000:
        bubble_data = bubble_data[bubble_data['artist'] != artist]

In [27]:
fig = px.scatter(bubble_data, x="review_count", y="score", animation_frame="year",size="views",title='', color="artist", hover_name="artist",range_x=[0,max(bubble_data['review_count'])+1],range_y=[-1,11],size_max=45)
fig.update_layout(showlegend=False, paper_bgcolor= '#F1EF80', plot_bgcolor= '#F1EF80',title='"The Needle Drop" - Most Successful Artists Timeseries')
fig.update_xaxes(showgrid=False,zerolinecolor='black',zerolinewidth=0.2)
fig.update_yaxes(showgrid=False,zerolinecolor='black',zerolinewidth=0.2)
fig.show()